In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

movies = None
ratings = None

In [ ]:
def compute_cosine_similarity(data, col_index, columns):
    # Create a pivot table with 'rating' values, Fill missing values with row means ,Compute cosine similarity between rows
    matrix = data.pivot(index=col_index, columns=columns, values='rating')
    matrix = matrix.apply(lambda row: row.fillna(row.mean()), axis=1)
    similarity_matrix = cosine_similarity(matrix)
    similarity_df = pd.DataFrame(similarity_matrix, index=matrix.index, columns=matrix.index)
    return similarity_df

In [ ]:
def compute_user_similarity(user_similarity_matrix = None):
    # If user_similarity_matrix is already computed, return it
    if user_similarity_matrix is not None:
        return user_similarity_matrix
    # Compute user similarity matrix using cosine similarity
    user_similarity_matrix = compute_cosine_similarity(ratings, 'userId', 'movieId')
    return user_similarity_matrix

In [ ]:
def compute_movie_similarity(movie_similarity_matrix = None):
    # If movie_similarity_matrix is already computed, return it
    if movie_similarity_matrix is not None:
        return  movie_similarity_matrix
    # Compute movie similarity matrix using cosine similarity
    movie_similarity_matrix =  compute_cosine_similarity(ratings, 'movieId', 'userId')
    return  movie_similarity_matrix

In [ ]:
def compute_most_similar_movies(movie, n, movie_similarity_matrix = None):
    # Compute movie similarity matrix if not provided
    movie_similarity_matrix = compute_movie_similarity()
    # Get the top similar movies for the given movie
    top_similar_movies = movie_similarity_matrix[movie].sort_values(ascending=False)[1:n+1]
    return movies[movies['movieId'].isin(top_similar_movies.index)]['title']

In [ ]:
def compute_recommended_movies(userId, movies_n,user_similarities_threshold,rating_threshold, top_n, user_similarity_matrix = None):
    # Compute user similarity matrix if not provided, Get user similarities for the given user
    user_similarity_matrix = compute_user_similarity()
    user_similarity = user_similarity_matrix[userId].sort_values(ascending=True)
    # Filter user similarities based on a threshold & Exclude the given user
    user_similarity = user_similarity[user_similarity > user_similarities_threshold]
    user_similarity = user_similarity[user_similarity.index != userId]
    # Select a subset of similar users
    user_similarity = user_similarity[:top_n]
    user_similarity = user_similarity.index

    # Get movies to recommend based on the similar users
    movies_to_recommend = ratings[ratings['userId'].isin(user_similarity)]
    movies_to_recommend = movies_to_recommend.groupby('movieId').mean()

    # Filter movies with rating above a threshold
    movies_to_recommend = movies_to_recommend[movies_to_recommend['rating'] > rating_threshold]
    movies_to_recommend = movies_to_recommend[:movies_n]

    # Sort recommended movies
    movies_to_recommend = movies_to_recommend.sort_values(by='rating', ascending=False)
    movies_to_recommend = movies_to_recommend.index

    return movies[movies['movieId'].isin(movies_to_recommend)]['title']

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

movies = movies[:200]
ratings = ratings[ratings['userId'] <= 200]
ratings = ratings[ratings['movieId'].isin(movies['movieId'])]


In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
195,229,Death and the Maiden (1994),Drama|Thriller
196,230,Dolores Claiborne (1995),Drama|Thriller
197,231,Dumb & Dumber (Dumb and Dumber) (1994),Adventure|Comedy
198,232,Eat Drink Man Woman (Yin shi nan nu) (1994),Comedy|Drama|Romance


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
28942,200,104,4.5,1229887243
28943,200,110,4.5,1229886082
28944,200,150,4.0,1229885978
28945,200,165,4.0,1229886183


Q1

In [ ]:
compute_most_similar_movies(1, 10)

8                                    Sudden Death (1995)
25                                        Othello (1995)
28     City of Lost Children, The (Cité des enfants p...
41                                     To Die For (1995)
51                          Home for the Holidays (1995)
73        Things to Do in Denver When You're Dead (1995)
77                                   White Squall (1996)
113                                Down Periscope (1996)
146                                     Jury Duty (1995)
173                               Unstrung Heroes (1995)
Name: title, dtype: object

In [ ]:
compute_movie_similarity()

movieId,1,2,3,4,5,6,7,8,9,10,...,223,224,225,227,228,229,230,231,232,233
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.985681,0.988021,0.988993,0.987811,0.985694,0.983688,0.987729,0.989088,0.978717,...,0.984225,0.988204,0.985939,0.984156,0.988993,0.988727,0.988386,0.970316,0.988784,0.988829
2,0.985681,1.000000,0.993404,0.994423,0.993530,0.990628,0.989449,0.992440,0.994310,0.984488,...,0.991085,0.993100,0.992134,0.990585,0.994423,0.994078,0.992848,0.979045,0.994154,0.994257
3,0.988021,0.993404,1.000000,0.997067,0.998057,0.993883,0.993400,0.996011,0.996735,0.987776,...,0.993608,0.995588,0.995712,0.993693,0.997067,0.996721,0.996360,0.981130,0.996797,0.996901
4,0.988993,0.994423,0.997067,1.000000,0.997290,0.996965,0.993447,0.997540,0.999887,0.990602,...,0.993343,0.998900,0.997537,0.996463,1.000000,0.999653,0.998416,0.983124,0.999730,0.999833
5,0.987811,0.993530,0.998057,0.997290,1.000000,0.994318,0.994183,0.996200,0.997177,0.988048,...,0.991751,0.995738,0.995973,0.993566,0.997290,0.996944,0.997010,0.980908,0.996860,0.997124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,0.988727,0.994078,0.996721,0.999653,0.996944,0.996619,0.993216,0.997194,0.999540,0.990258,...,0.993712,0.998679,0.997191,0.996117,0.999653,1.000000,0.998268,0.982783,0.999383,0.999486
230,0.988386,0.992848,0.996360,0.998416,0.997010,0.995997,0.992550,0.995960,0.998303,0.989259,...,0.992612,0.997030,0.996382,0.994727,0.998416,0.998268,1.000000,0.980173,0.998073,0.998329
231,0.970316,0.979045,0.981130,0.983124,0.980908,0.980304,0.978764,0.980514,0.983054,0.978688,...,0.972981,0.981886,0.982297,0.980277,0.983124,0.982783,0.980173,1.000000,0.982858,0.982989


In [ ]:
compute_most_similar_movies(4, 10)

48                                       Lamerica (1994)
50                                        Georgia (1995)
54                                 Eye for an Eye (1996)
57                                  Two if by Sea (1996)
63                                      Fair Game (1995)
64                          Kicking and Screaming (1995)
151                                      Mad Love (1995)
158                                    Party Girl (1995)
160                                      Reckless (1995)
168    Umbrellas of Cherbourg, The (Parapluies de Che...
Name: title, dtype: object

In [ ]:
# Recommend 5 movies for user 200 with a user similarity threshold of 0.3 and rating threshold of 4.5, considering top 20 movies
print(compute_recommended_movies(200, 5, 0.3,4.5, 20))

74          Antonia's Line (Antonia) (1995)
76                Angels and Insects (1995)
87     Heidi Fleiss: Hollywood Madam (1995)
108           Flirting With Disaster (1996)
121        Awfully Big Adventure, An (1995)
Name: title, dtype: object
